In [14]:
import matplotlib.pyplot as plt

In [7]:
from numpy.polynomial.laguerre import laggauss

np.version.version

'1.16.4'

In [11]:
from scipy.special import roots_legendre

roots_legendre(10)

(array([-0.97390653, -0.86506337, -0.67940957, -0.43339539, -0.14887434,
         0.14887434,  0.43339539,  0.67940957,  0.86506337,  0.97390653]),
 array([0.06667134, 0.14945135, 0.21908636, 0.26926672, 0.29552422,
        0.29552422, 0.26926672, 0.21908636, 0.14945135, 0.06667134]))

\begin{equation}
r_i = \sqrt{x_i^2 + y_i^2 + z_i^2} 
\end{equation}

In [34]:
# %load legendre.py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import jit, njit, prange
from numpy.polynomial.legendre import leggauss


N = 30

x,w = leggauss(N)
a = -2
b = 2


@njit(paraellel=True)
def integrate(x,a,b, alpha=2):
    totalSum = 0
    xl = (b+a)*0.5
    xm = (b-a)*0.5 
    for i in prange(N):
        for j in prange(N):
            for k in prange(N):
                for l in prange(N):
                    for m in prange(N):
                        for n in prange(N):
                            x1, y1, z1, x2, y2, z2 = xm*x[i]+xl, xm*x[j]+xl, xm*x[k]+xl, xm*x[l]+xl, xm*x[m]+xl, xm*x[n]+xl
                            exp1 = -2*alpha*np.sqrt(x1*x1+ y1*y1 + z1*z1)
                            exp2 = -2*alpha*np.sqrt(x2*x2+y2*y2+z2*z2)
                            deno = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
                            weight = w[i]*w[j]*w[k]*w[l]*w[m]*w[n]

                            if deno > 10**-10:
                                totalSum += weight*np.exp(exp1+ exp2)/deno
        print(i)
    return totalSum*((b-a)*0.5)**6


result =integrate(x,a=-2,b=2)
analytical = 5*np.pi**2/16**2
error = analytical-result
print('Analytical: {:.5f}'.format(analytical))
print('Result: {:.5f}'.format(result))
print('Error: {:.5f}'.format(error))

/home/ovewh/anaconda3/lib/python3.7/site-packages/numba/decorators.py:220: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Analytical: 0.19277
Result: 0.18580
Error: 0.00697


In [33]:
# %load laguerre.py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import njit, prange
from numpy.polynomial.legendre import leggauss
from scipy.special import roots_laguerre



N =40
x,wga  = roots_laguerre(N)

leg, wge = leggauss(N)

@njit(parallel=True)
def integrate(x, leg, wga, wge):
    totalSum = 0
    thetaScalem = np.pi/2
    pScalem = 2*np.pi/2 
    for i in prange(N):
        for j in prange(N):
            for k in prange(N):
                for l in prange(N):
                    for m in prange(N):
                        for n in prange(N):
                            r1, t1, phi1 = x[i], thetaScalem*leg[j] + thetaScalem, pScalem*leg[k] +pScalem
                            r2, t2, phi2 = x[l], thetaScalem*leg[m] + thetaScalem, pScalem*leg[n] +pScalem

                            cosbeta = np.cos(t1)*np.cos(t2) + (np.sin(t1)*np.sin(t2)*np.cos(phi1-phi2))
                            func = np.exp(-3*(r1+r2))*r1*r1*r2*r2*np.sin(t1)*np.sin(t2)
                            denom = r1*r1+r2*r2-(2*r1*r2*cosbeta)
                            if denom > 1e-10:
                                weight =  wga[i]*wga[l]*wge[j]*wge[m]*wge[k]*wge[n]
                                func = func/np.sqrt(denom)
                                totalSum += func*weight
        print(i)
    return pScalem**2*thetaScalem**2*totalSum


result = integrate(x,leg, wga, wge)
analytical = 5*np.pi**2/16**2
error = analytical - result

print('Analytical: {:.5f}'.format(analytical))
print('Result: {:.5f}'.format(result))
print('Error: {:.5f}'.format(error))


10
0
30
20
11
1
21
31
12
2
22
32
13
3
23
33
14
4
24
34
15
5
25
35
16
6
26
36
17
7
27
37
18
8
28
19
38
9
29
39
Analytical: 0.19277
Result: 0.19467
Error: -0.00190


We see that $e^{-2\alpha r_i} \approx 0$ when $\lambda = 7$ 